In [19]:
import datetime
import pandas as pd
import numpy as np 

import glob
# # https://pypi.org/project/bng-latlon/
from bng_to_latlon import OSGB36toWGS84

In [ ]:
# new london dataset `: https://data.london.gov.uk/dataset/allotment-locations
# locations
# https://github.com/zzsza/Awesome-Mobility-Machine-Learning-Contents#route-planning
# https://github.com/awesomedata/awesome-public-datasets#transportation

In [ ]:
# http://www.nationalarchives.gov.uk/doc/open-government-licence/version/2/
# https://data.london.gov.uk/dataset/gla-group-land-assets?q=gla%20asset


In [20]:
# https://stackoverflow.com/questions/20906474/import-multiple-csv-files-into-pandas-and-concatenate-into-one-dataframe
path = '/Users/qiyang/Documents/qduan/temp/london_addr' # use your path
all_files = glob.glob(path + "/orig/*.csv")

all_files 

['/Users/qiyang/Documents/qduan/temp/london_addr/orig/GLA_assets.csv',
 '/Users/qiyang/Documents/qduan/temp/london_addr/orig/TFL_assets.csv',
 '/Users/qiyang/Documents/qduan/temp/london_addr/orig/LFB_assets.csv',
 '/Users/qiyang/Documents/qduan/temp/london_addr/orig/LLDC_assets.csv',
 '/Users/qiyang/Documents/qduan/temp/london_addr/orig/Met_assets.csv']

In [21]:
filename =  '/Users/qiyang/Documents/qduan/temp/london_addr/orig/GLA_assets.csv'
filename.split("/")[-1].split("_")[0]

'GLA'

In [22]:

li = []

for filename in all_files: # [1:2]
    print(filename)
    visit = pd.read_csv(filename, 
                    header = 0,
                    delimiter=',',
                    encoding = "ISO-8859-1")
    if visit.shape[0] < 1 :
        continue
    print(visit.count().max())
    visit  = visit [visit['Easting'].notnull()].copy()
    
    visit['long'] = visit.apply(lambda x: OSGB36toWGS84(  x['Easting'], x['Northing']) [1] , axis=1)
    visit['lat'] = visit.apply(lambda x: OSGB36toWGS84(  x['Easting'], x['Northing']) [0] , axis=1)
    visit['new_id'] = visit.apply(lambda x: '_'.join( [ 
        filename.split("/")[-1].split("_")[0] ,
        str(x['Unique_Asset_ID'])  
        ]), axis=1)

    
    print(visit.count().max())
    li.append(visit)

dframe = pd.concat(li, axis=0, ignore_index=True)
print(dframe.count().max())
dframe[['Address', 'Easting', 'Northing', 'long', 'lat', 'new_id']]


/Users/qiyang/Documents/qduan/temp/london_addr/orig/GLA_assets.csv
162
162
/Users/qiyang/Documents/qduan/temp/london_addr/orig/TFL_assets.csv
656
654
/Users/qiyang/Documents/qduan/temp/london_addr/orig/LFB_assets.csv
116
115
/Users/qiyang/Documents/qduan/temp/london_addr/orig/LLDC_assets.csv
29
29
/Users/qiyang/Documents/qduan/temp/london_addr/orig/Met_assets.csv
302
302
1262


/Users/qiyang/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:25: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



Address   Easting  Northing  \
0     Crystal Palace Park, Ledrington Road, Crystal ...  534361.0  170764.0   
1     Crystal Palace Park, Ledrington Road, Crystal ...  534225.0  170850.0   
2     Crystal Palace Park, Ledrington Road, Crystal ...  534339.0  170472.0   
3     Crystal Palace Park, Ledrington Road, Crystal ...  534345.0  170981.0   
4                      Coldharbour Lane, Rainham, Essex  551802.0  180486.0   
...                                                 ...       ...       ...   
1257  SAINSBURY'S SUPERMARKET GILLINGHAM STREET 99 W...  529107.0  178768.0   
1258  WEST END CENTRAL STATION 27 SAVILE ROW . WESTM...  529081.0  180818.0   
1259  WESTMINSTER (FORMER) HEADQUARTERS VICTORIA EMB...  530282.0  179871.0   
1260                                   LONDON, LONDON,   526997.0  182077.0   
1261                                    LONDON, LONDON,  526997.0  182077.0   

          long        lat      new_id  
0    -0.069065  51.419973  GLA_02/001  
1    -0.070987  51.420778  GLA_02/002  
2    -0.069492  51.417354  GLA_02/003  
3    -0.069213  51.421927  GLA_02/004  
4     0.185750  51.502927       GLA_0  
...        ...        ...         ...  
1257 -0.141663  51.493124    Met_6641  
1258 -0.141288  51.511552     Met_238  
1259 -0.124341  51.502767     Met_103  
1260 -0.170850  51.523339    Met_6082  
1261 -0.170850  51.523339    Met_6083  

[1262 rows x 6 columns]

In [35]:
dframe['Site_Area_Hectares'].fillna(0, inplace=True)   
dframe['Building_Area_GIA_sqm'].fillna(0, inplace=True)   
# dframe['overall_hectares'] = dframe['Site_Area_Hectares']
dframe['overall_hectares'] = dframe.apply(lambda x:   x['Site_Area_Hectares'] if  x['Site_Area_Hectares'] > 0 else x['Building_Area_GIA_sqm'] / 10000 , axis=1)
# dframe['overall_hectares'] = dframe.apply(lambda x:   x['Site_Area_Hectares']  + ( x['Building_Area_GIA_sqm'] / 10000 ), axis=1)

In [23]:
dframe.columns

Index(['Address', 'Asset_Category', 'Borough', 'Building_Area_GIA_sqm',
       'Building_Area_NIA_sqm', 'Built_out', 'Description', 'Easting',
       'Field18', 'Field19', 'Field20', 'Field21', 'Field22', 'Field23',
       'Field24', 'Field25', 'Field26', 'Field27', 'Field28', 'Field29',
       'Holding_Name', 'Land_Building', 'Land_Buildings', 'Northing',
       'OBJECTID', 'Occupied', 'Occupied_by', 'Occupied_by_LLDC',
       'Occupied_by_MPS', 'Occupied_by_TfL', 'Post_Code', 'Site_Area_Hectares',
       'Site_Area_Hectares_1', 'Sub_Unit', 'Tenure', 'UPRN', 'Unique_Asset_ID',
       'Unique_Asset_ID_X', 'Unique_Asset_ID_Y', 'lat', 'long', 'new_id'],
      dtype='object')

In [34]:
dframe[dframe['new_id']  == 'Met_103'][['new_id', 'Easting', 'Northing', 'lat', 'long', 'Address', 'Post_Code', 
        'overall_hectares', 'Site_Area_Hectares', 'Building_Area_GIA_sqm','Site_Area_Hectares_1',]]

new_id   Easting  Northing        lat      long  \
1259  Met_103  530282.0  179871.0  51.502767 -0.124341   

                                                Address Post_Code  \
1259  WESTMINSTER (FORMER) HEADQUARTERS VICTORIA EMB...  SW1A 2JL   

      overall_hectares  Site_Area_Hectares  Building_Area_GIA_sqm  \
1259              0.23                0.23                 8691.0   

      Site_Area_Hectares_1  
1259                   NaN

In [36]:
dframe[['new_id', 'Easting', 'Northing', 'lat', 'long', 'Address', 'Post_Code', 
        'overall_hectares', 'Site_Area_Hectares', 'Building_Area_GIA_sqm','Site_Area_Hectares_1',
        'OBJECTID', 'Borough', 'Unique_Asset_ID', 'Unique_Asset_ID_X',
       'Unique_Asset_ID_Y', 'Holding_Name', 'Sub_Unit',
       'UPRN', 'Description', 'Occupied', 'Land_Building', 'Asset_Category',
       'Tenure']
      ].to_csv ( path+r'/all_addreses.csv',
                index = None, header=True, 
                sep='|',
                encoding = "utf-8") #Don't forget to add '.csv' at the end of the path

In [38]:
[dframe[  'lat'].max(), dframe[  'long'].min()]

[51.70507024852442, -0.8049858046195676]

In [39]:
[dframe[  'lat'].min(), dframe[  'long'].max()]

[51.30832941013647, 0.3297512620904363]